In [1]:
import logging
import random

import numpy as np

import gym
from gym import wrappers

logging.basicConfig(level=logging.INFO)

# ACS2 in *FrozenLake*

> The agent controls the movement of a character in a grid world. Some tiles of the grid are walkable, and others lead to the agent falling into the water. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile.

In [2]:
env = gym.make('FrozenLake-v0')

# Reset the state
state = env.reset()

# Render the environment
env.render()

INFO:gym.envs.registration:Making new env: FrozenLake-v0



SFFF
FHFH
FFFH
HFFG


Each state might get following possible values: `{S, F, H, G}` which, refers to
```
SFFF       (S: starting point, safe)
FHFH       (F: frozen surface, safe)
FFFH       (H: hole, fall to your doom)
HFFG       (G: goal, where the frisbee is located)
```

In case of interacting with environment agent cant perform 4 action which map as follow:
- 0 - left
- 1 - down
- 2 - right
- 3 - up

> FrozenLake-v0 defines "solving" as getting average reward of 0.78 over 100 consecutive trials.

## Setting up the Agent
Unfortunatelly the only information back from the environment is the current agent position. Therefore our agent task will be to predicit where it will land after executing each action.

To do so we will represent state as a one-hot encoded vector.

In [3]:
# Import PyALCS code from local path
import sys
sys.path.append('/Users/khozzy/Projects/pyalcs')

from alcs import ACS2, ACS2Configuration

# Enable automatic module reload
%load_ext autoreload
%autoreload 2

# Set some constants
CLASSIFIER_LENGTH = 16
POSSIBLE_ACTIONS = env.action_space.n

In [4]:
def one_hot_encode(state):
    vec = ['0' for i in range(CLASSIFIER_LENGTH)]
    vec[state] = 'X'
    return ''.join(vec)

`X` corresponds to current agent position. State 4 is encoded as follows:

In [5]:
one_hot_encode(4)

'0000X00000000000'

We will also need a function for evaluating if agent finished succesfuly a trial

In [6]:
# We assume if the final state was with number 15 that the algorithm found the reward. Otherwise not
def collect_env_metrics(env):
    state = None
    
    if type(env) is gym.wrappers.monitoring.Monitor:
        state = env.env.env.s
    else:
        state = env.env.s
        
    return {'found_reward': state == 15}

Right now we are ready to configure the ACS2 agent

In [7]:
cfg = ACS2Configuration(
    classifier_length=CLASSIFIER_LENGTH,
    number_of_possible_actions=POSSIBLE_ACTIONS,
    perception_mapper_fcn=one_hot_encode,
    environment_metrics_fcn=collect_env_metrics,
    theta_i=0.3,
    epsilon=0.7)

print(cfg)

ACS2Configuration:
	- Classifier length: [16]
	- Number of possible actions: [4]
	- Classifier wildcard: [#]
	- Perception mapper function: [<function one_hot_encode at 0x112260950>]
	- Action mapping dict: [None]
	- Environment metrics function: [<function collect_env_metrics at 0x112260d90>]
	- Performance calculation function: [None] 
	- Do GA: [False]
	- Do subsumption: [True]
	- Beta: [0.05]
	- ...
	- Epsilon: [0.7]
	- U_max: [100000]


Build agent using defined configuration

In [ ]:
agent = ACS2(cfg)

Learn some behaviour during exploration phase

In [ ]:
EXPLOIT_TRIALS = 1000

population, metrics = agent.explore(env, EXPLOIT_TRIALS)

INFO:root:{'agent': {'population': 5, 'numerosity': 5, 'reliable': 0, 'fitness': 0.23862812500000002, 'trial': 0, 'steps': 5, 'total_steps': 5}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 11, 'numerosity': 11, 'reliable': 0, 'fitness': 0.23520911603116124, 'trial': 1, 'steps': 6, 'total_steps': 11}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 11, 'numerosity': 11, 'reliable': 0, 'fitness': 0.2322538637229226, 'trial': 2, 'steps': 2, 'total_steps': 13}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 13, 'numerosity': 13, 'reliable': 0, 'fitness': 0.23190282441390206, 'trial': 3, 'steps': 7, 'total_steps': 20}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 14, 'numerosity': 14, 'reliable': 0, 'fitness': 0.2309750856849897, 'trial': 4, 'steps': 6, 'total_steps': 26}, 'environment': {'foun

INFO:root:{'agent': {'population': 112, 'numerosity': 112, 'reliable': 0, 'fitness': 0.1661555187515018, 'trial': 39, 'steps': 14, 'total_steps': 252}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 113, 'numerosity': 113, 'reliable': 0, 'fitness': 0.1658927713557469, 'trial': 40, 'steps': 2, 'total_steps': 254}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 112, 'numerosity': 112, 'reliable': 0, 'fitness': 0.16410206360213228, 'trial': 41, 'steps': 11, 'total_steps': 265}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 113, 'numerosity': 113, 'reliable': 0, 'fitness': 0.1621712946809151, 'trial': 42, 'steps': 7, 'total_steps': 272}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 118, 'numerosity': 118, 'reliable': 0, 'fitness': 0.1615340537239816, 'trial': 43, 'steps': 10, 'total_steps': 282

INFO:root:{'agent': {'population': 147, 'numerosity': 147, 'reliable': 0, 'fitness': 0.137661799057557, 'trial': 78, 'steps': 2, 'total_steps': 497}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 147, 'numerosity': 147, 'reliable': 0, 'fitness': 0.13716019530755558, 'trial': 79, 'steps': 4, 'total_steps': 501}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 148, 'numerosity': 148, 'reliable': 0, 'fitness': 0.13700569741292, 'trial': 80, 'steps': 4, 'total_steps': 505}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 147, 'numerosity': 147, 'reliable': 0, 'fitness': 0.13768274715696885, 'trial': 81, 'steps': 12, 'total_steps': 517}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 147, 'numerosity': 147, 'reliable': 0, 'fitness': 0.1375749931007494, 'trial': 82, 'steps': 7, 'total_steps': 524}, '

INFO:root:{'agent': {'population': 168, 'numerosity': 168, 'reliable': 0, 'fitness': 0.13256295228576806, 'trial': 117, 'steps': 8, 'total_steps': 798}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 172, 'numerosity': 172, 'reliable': 0, 'fitness': 0.1334790313515567, 'trial': 118, 'steps': 10, 'total_steps': 808}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 173, 'numerosity': 173, 'reliable': 0, 'fitness': 0.13367452230571153, 'trial': 119, 'steps': 3, 'total_steps': 811}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 177, 'numerosity': 177, 'reliable': 0, 'fitness': 0.13594511558738026, 'trial': 120, 'steps': 8, 'total_steps': 819}, 'environment': {'found_reward': True}, 'performance': None}
INFO:root:{'agent': {'population': 179, 'numerosity': 179, 'reliable': 0, 'fitness': 0.13626577201730466, 'trial': 121, 'steps': 4, 'total_steps'

INFO:root:{'agent': {'population': 196, 'numerosity': 196, 'reliable': 0, 'fitness': 0.1370167620461836, 'trial': 156, 'steps': 12, 'total_steps': 1045}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 200, 'numerosity': 200, 'reliable': 0, 'fitness': 0.1376433805983334, 'trial': 157, 'steps': 13, 'total_steps': 1058}, 'environment': {'found_reward': True}, 'performance': None}
INFO:root:{'agent': {'population': 204, 'numerosity': 204, 'reliable': 0, 'fitness': 0.13610960435129146, 'trial': 158, 'steps': 7, 'total_steps': 1065}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 204, 'numerosity': 204, 'reliable': 0, 'fitness': 0.13569838493620404, 'trial': 159, 'steps': 5, 'total_steps': 1070}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 203, 'numerosity': 203, 'reliable': 0, 'fitness': 0.13555444654802246, 'trial': 160, 'steps': 16, 'total_s

...

INFO:root:{'agent': {'population': 316, 'numerosity': 316, 'reliable': 0, 'fitness': 0.10061290573204978, 'trial': 390, 'steps': 2, 'total_steps': 2876}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 315, 'numerosity': 315, 'reliable': 0, 'fitness': 0.10015307210864893, 'trial': 391, 'steps': 7, 'total_steps': 2883}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 316, 'numerosity': 316, 'reliable': 0, 'fitness': 0.10012249610995244, 'trial': 392, 'steps': 8, 'total_steps': 2891}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 320, 'numerosity': 320, 'reliable': 0, 'fitness': 0.10083442647723788, 'trial': 393, 'steps': 4, 'total_steps': 2895}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 319, 'numerosity': 319, 'reliable': 0, 'fitness': 0.10128128059249555, 'trial': 394, 'steps': 12, 'total_

INFO:root:{'agent': {'population': 327, 'numerosity': 327, 'reliable': 0, 'fitness': 0.10209020405679521, 'trial': 428, 'steps': 11, 'total_steps': 3207}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 327, 'numerosity': 327, 'reliable': 0, 'fitness': 0.10201438917980872, 'trial': 429, 'steps': 3, 'total_steps': 3210}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 328, 'numerosity': 328, 'reliable': 0, 'fitness': 0.10158878250569131, 'trial': 430, 'steps': 12, 'total_steps': 3222}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 328, 'numerosity': 328, 'reliable': 0, 'fitness': 0.10107489701278152, 'trial': 431, 'steps': 10, 'total_steps': 3232}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 328, 'numerosity': 328, 'reliable': 0, 'fitness': 0.10082454410652344, 'trial': 432, 'steps': 5, 'tota

INFO:root:{'agent': {'population': 334, 'numerosity': 334, 'reliable': 0, 'fitness': 0.09540911890101708, 'trial': 467, 'steps': 5, 'total_steps': 3530}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 333, 'numerosity': 333, 'reliable': 0, 'fitness': 0.09463288830773442, 'trial': 468, 'steps': 13, 'total_steps': 3543}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 320, 'numerosity': 320, 'reliable': 0, 'fitness': 0.09622979619900675, 'trial': 469, 'steps': 5, 'total_steps': 3548}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 316, 'numerosity': 316, 'reliable': 0, 'fitness': 0.09634398994403055, 'trial': 470, 'steps': 9, 'total_steps': 3557}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 316, 'numerosity': 316, 'reliable': 0, 'fitness': 0.09622668767883728, 'trial': 471, 'steps': 2, 'total_

INFO:root:{'agent': {'population': 307, 'numerosity': 307, 'reliable': 0, 'fitness': 0.08967516366118539, 'trial': 506, 'steps': 6, 'total_steps': 3860}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 309, 'numerosity': 309, 'reliable': 0, 'fitness': 0.08963822627041294, 'trial': 507, 'steps': 7, 'total_steps': 3867}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 309, 'numerosity': 309, 'reliable': 0, 'fitness': 0.08944613581372877, 'trial': 508, 'steps': 2, 'total_steps': 3869}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 309, 'numerosity': 309, 'reliable': 0, 'fitness': 0.08996395432779374, 'trial': 509, 'steps': 2, 'total_steps': 3871}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 313, 'numerosity': 313, 'reliable': 0, 'fitness': 0.0905810595124196, 'trial': 510, 'steps': 13, 'total_s

INFO:root:{'agent': {'population': 340, 'numerosity': 340, 'reliable': 0, 'fitness': 0.09659534731525929, 'trial': 545, 'steps': 4, 'total_steps': 4185}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 340, 'numerosity': 340, 'reliable': 0, 'fitness': 0.09640720751602432, 'trial': 546, 'steps': 2, 'total_steps': 4187}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 341, 'numerosity': 341, 'reliable': 0, 'fitness': 0.09640433506552973, 'trial': 547, 'steps': 5, 'total_steps': 4192}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 341, 'numerosity': 341, 'reliable': 0, 'fitness': 0.09631284456988426, 'trial': 548, 'steps': 2, 'total_steps': 4194}, 'environment': {'found_reward': False}, 'performance': None}
INFO:root:{'agent': {'population': 342, 'numerosity': 342, 'reliable': 0, 'fitness': 0.0964445688598323, 'trial': 549, 'steps': 3, 'total_st

In [ ]:
population.sort(key=lambda cl: -cl.fitness)
population_count = len(population)
reliable_count = len([cl for cl in population if cl.is_reliable()])
successful_trials = sum(m['environment']['found_reward'] for m in metrics)

print("Number of classifiers: {}".format(population_count))
print("Number of reliable classifiers: {}".format(reliable_count))
print("Percentage of successul trials: {:.2f}%".format(successful_trials / EXPLOIT_TRIALS  * 100))
print("\nTop 10 classifiers:")
for cl in population[:10]:
    print("{!r} \tq: {:.2f} \tr: {:.2f} \tir: {:.2f} \texp: {}".format(cl, cl.q, cl.r, cl.ir, cl.exp))

Now let's try to reuse this knowledge - exploitation phase

In [ ]:
TEST_TRIALS = 100

# Create new agent with initial classifiers population
exploiter = ACS2(cfg, population=population)

# Record performance, force erase previous run
env = wrappers.Monitor(env, "/tmp/gym-results", force=True)

_, metrics = exploiter.exploit(env, TEST_TRIALS)

env.close()

In [ ]:
successful_trials = sum(m['environment']['found_reward'] for m in metrics)

print("Percentage of successul trials: {:.2f}%".format(successful_trials / TEST_TRIALS * 100))